In [1]:
#Path to Images
impath = "data/images_training_rev1"
#Path to Labels CSV
lapath = "training_solutions_rev1.csv"
#N test images
ntest = 10000
#Resolution
pixres = 420

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import skimage.io as io
import tensorflow as tf
import pathlib
import os
import sys
%matplotlib inline

In [3]:
label_df = pd.read_csv(lapath)
label_df["GalaxyID"] = label_df["GalaxyID"].apply((lambda x: str(x) + ".jpg"))
tests = np.random.choice(range(len(label_df)), ntest, replace=False)
testdf = label_df.iloc[tests]
traindf = label_df.drop(index=tests)

datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.1)
testgen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=impath,
x_col="GalaxyID",
y_col=['Class1.1','Class1.2','Class1.3'],
class_mode = "raw",
subset="training",
batch_size=32,
color_mode="grayscale",
interpolation='bilinear',
target_size=(pixres,pixres))

validation_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=impath,
x_col="GalaxyID",
y_col=['Class1.1','Class1.2','Class1.3'],
class_mode = "raw",
subset="validation",
batch_size=32,
color_mode="grayscale",
interpolation='bilinear',
target_size=(pixres,pixres))

test_generator=testgen.flow_from_dataframe(
dataframe=testdf,
directory=impath, 
x_col='GalaxyID',
y_col=['Class1.1','Class1.2','Class1.3'],
target_size=(pixres,pixres),
color_mode="grayscale",
class_mode='raw',
interpolation='bilinear',
batch_size=1)

Found 46421 validated image filenames.
Found 5157 validated image filenames.
Found 10000 validated image filenames.


In [4]:
basic_rms_model = tf.keras.models.Sequential()
basic_rms_model.add(tf.keras.Input(shape=(pixres,pixres,1)))
basic_rms_model.add(tf.keras.layers.Flatten())
basic_rms_model.add(tf.keras.layers.Dense(32))
basic_rms_model.add(tf.keras.layers.Dense(32))
basic_rms_model.add(tf.keras.layers.Dense(3,activation='sigmoid'))
basic_rms_model.compile(
    loss='mean_squared_error',
    metrics=['acc']
)
#basic_rms_model.summary()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-05-27 17:29:45.967040: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-27 17:29:45.967186: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
basic_conv2d_model = tf.keras.models.Sequential()
basic_conv2d_model.add(tf.keras.Input(shape=(pixres,pixres,1)))
basic_conv2d_model.add(tf.keras.layers.Conv2D(32,(3,3),activation="relu"))
basic_conv2d_model.add(tf.keras.layers.Conv2D(64,(3,3),activation="relu"))
basic_conv2d_model.add(tf.keras.layers.Flatten())
basic_conv2d_model.add(tf.keras.layers.Dense(32))
basic_conv2d_model.add(tf.keras.layers.Dense(3,activation='sigmoid'))
basic_conv2d_model.compile(
    loss='mean_squared_error',
    metrics=['acc']
)
#basic_conv2d_model.summary()

In [6]:
conv_pool_model = tf.keras.models.Sequential()
conv_pool_model.add(tf.keras.Input(shape=(pixres,pixres,1)))
conv_pool_model.add(tf.keras.layers.CenterCrop(207, 207))
conv_pool_model.add(tf.keras.layers.MaxPool2D(pool_size=3))
conv_pool_model.add(tf.keras.layers.Conv2D(32,(3,3),activation="relu"))
conv_pool_model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
conv_pool_model.add(tf.keras.layers.Conv2D(64,(3,3),activation="relu"))
conv_pool_model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
conv_pool_model.add(tf.keras.layers.Flatten())
conv_pool_model.add(tf.keras.layers.Dense(32))
conv_pool_model.add(tf.keras.layers.Dense(3,activation='sigmoid'))
conv_pool_model.compile(
    loss='mean_squared_error',
    optimizer='SGD',
    metrics=['acc']
)
conv_pool_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 center_crop (CenterCrop)    (None, 207, 207, 1)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 69, 69, 1)        0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 67, 67, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 33, 33, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 31, 31, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 64)      

In [7]:
conv_pool_model.fit(x=train_generator,epochs=15,validation_data=validation_generator,validation_freq=1,verbose=1)
conv_pool_score = conv_pool_model.evaluate(test_generator, return_dict=True)
print(conv_pool_score)

Epoch 1/15


2022-05-27 17:29:46.502544: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-27 17:29:46.826826: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1451/1451 [==============================] - ETA: 0s - loss: 0.0368 - acc: 0.7535

2022-05-27 17:31:38.463171: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1451/1451 [==============================] - 124s 85ms/step - loss: 0.0368 - acc: 0.7535 - val_loss: 0.0387 - val_acc: 0.6814
Epoch 2/15
1451/1451 [==============================] - 124s 86ms/step - loss: 0.0263 - acc: 0.8091 - val_loss: 0.0221 - val_acc: 0.8249
Epoch 3/15
1451/1451 [==============================] - 125s 86ms/step - loss: 0.0233 - acc: 0.8201 - val_loss: 0.0234 - val_acc: 0.8131
Epoch 4/15
1451/1451 [==============================] - 125s 86ms/step - loss: 0.0217 - acc: 0.8280 - val_loss: 0.0221 - val_acc: 0.8214
Epoch 5/15
1008/1451 [===================>..........] - ETA: 33s - loss: 0.0207 - acc: 0.8306

KeyboardInterrupt: 

In [ ]:
conv_pool_model.save('conv_pool.h5')